In [ ]:
#def func_gradient(func, theta_sym, values):
#    grad = np.array([diff(L(*theta_sym), i) for i in theta_sym])
#    substitutes = list(zip(theta_sym, values))
#
#    for i, element in enumerate(grad):
#        for j in substitutes:
#            grad[i] = grad[i].subs(*j)
#    return grad  

In [6]:
def L(x,y):
    return x**2*y**2 + y**2

def L_gradient(x,y):
    return np.array([2*x*y**2, 2*x**2 * y + 2*y])


In [46]:
# from Algorithm 1: HMC, page1353
def Leapfrog(theta, r, eps):
    r_tilde = r + (eps/2) * L_gradient(*theta)
    theta_tilde = theta + eps * r_tilde
    r_tide = r_tilde + (eps/2) * L_gradient(*theta_tilde)
    return theta_tilde, r_tilde

In [47]:
theta = np.array([0.5,0.4])
r = np.array([0.5, 0.4])
u = 0.5
v = 0.5
eps = 0.5
Leapfrog(theta, r, eps)

(array([ 0.77 ,  0.725]), array([ 0.54,  0.65]))

In [17]:
def BuildTree(theta, r, u, v, j, eps):
    if j == 0:
        # base case, take one leapfrog step in the direction v
        theta_prime, r_prime = Leapfrog(theta, r, v*eps)
        
        if u <= np.exp(L(*theta_prime) - 0.5 * np.dot(r_prime, r_prime)):
            C_prime = np.append(theta_prime, r_prime)
        else:
            C_prime = np.empty(shape=(0, 0))
        
        s_prime = int(L(*theta_prime) - 0.5 * np.dot(r_prime, r_prime) > np.log(u) - 1000)
        
        return theta_prime, r_prime, theta_prime, r_prime, C_prime, s_prime
    else:
        # recursion, build left and right subtrees
        theta_minus, r_minus, theta_plus, r_plus, C_prime, s_prime = BuildTree(theta, r, u, v, j-1, eps)
        
        if v == -1:
            theta_minus, r_minus, _,_, C_doub_prime, s_doub_prime = BuildTree(theta_minus, r_minus, u, v, j-1, eps)
        else:
            _,_, theta_plus, r_plus, C_doub_prime, s_doub_prime = BuildTree(theta_plus, r_plus, u, v, j-1, eps)
            
        ind_1 = int(np.dot(theta_plus-theta_minus, r_minus) >= 0)
        ind_2 = int(np.dot(theta_plus-theta_minus, r_plus) >= 0)
        s_prime = s_prime * s_doub_prime * ind_1 * ind_2
        C_prime = np.append(C_prime, C_doub_prime)
        
        return theta_minus, r_minus, theta_plus, r_plus, C_prime, s_prime

In [ ]:
def naive_nuts(theta0, eps, L, M, L_gradient):
    for m in range(M):
        # resample
        r0 = np.random.normal(0,1,1)
        upper = np.exp(L(*theta**m) - 0.5*np.dot(r0, r0))
        u = np.random.uniform(0, upper, 1)
        
        # initialize
        theta_minus = theta**m 
        theta_plus = theta**m
        r_minus = r0
        r_plus = r0
        j = 0
        C = np.append(theta**m, r0)
        s = 1
        
        while s == 1:
            v_j = np.random.uniform(-1,1,1)
            if v_j == -1:
                theta_minus, r_minus, _,_, C_prime, s_prime = BuildTree(theta_minus, r_minus, u, v_j, j, eps)
            else:
                _,_, theta_plus, r_plus, C_prime, s_prime = BuildTree(theta_plus, r_plus, u, v_j, j, eps)
            
            if s_prime = 1:
                C = np.append(C, C_prime)
            
            boolean_1 = int
            boolean_2 =
            s = s_

In [42]:
theta = np.array([3,5])
theta
theta**2

def f(x, y):
    return x + y

f(*theta**2)

34

In [48]:
one = [1,2]
two = [3,4]
np.append(one,two)

array([1, 2, 3, 4])

In [ ]:
one = np.array([1,2])
two = np.array([3,4])
values

In [ ]:
np.append(one,two)

In [ ]:
esto = 2 > 3
1 * esto

In [ ]:
int(esto)

In [ ]:
2 * 3